# Google Maps

In [2]:
#Importamos las librerias que usaremos
import pandas as pd
import os
from datetime import datetime
import dask.dataframe as dd
import ast

C:\Users\LENOVO\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\dask\dataframe\__init__.py:31: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


## Metadata sitios
A continuacion se realizara la union de los distintos data sets de los datos de negocios y se filtraran por la categoria de restaurantes

In [2]:
ruta_entrada = '../Data/Data_limpia/Google/Metadata'
ruta_salida = '../Data/Data_transformada/Metadata'

# Creamos una lista vacia que va a almacenar todos los data frames 
dfs = []

        # Iteramos la carpeta para leer los archivos
for item in os.listdir(ruta_entrada):
     # Verificamos que sea un archivo JSON y guardamos su ruta
    if item.endswith('.json'):
        path_item = os.path.join(ruta_entrada,item)

        # Leemos el archivo JSON
        df = pd.read_json(path_item, lines = True)

        # Añadimos el DF a la lista de DFS
        dfs.append(df)

# Concatenamos los dataframes para tener un archivo unico      
df_combinado = pd.concat(dfs) 

# Definimos la ruta de salida
salida = os.path.join(ruta_salida, 'Sitios_combinada.parquet')

# Guardamos el data frame en formato parquet
df_combinado.to_parquet(salida, index = False)

In [3]:
# Leemos el archivo de forma porcionada ya que es un data set muy grande
df = dd.read_parquet('../Data/Data_transformada/Metadata/Sitios_combinada.parquet')

In [4]:
df.head(5)

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
0,Porter Pharmacy,"Porter Pharmacy, 129 N Second St, Cochran, GA ...",0x88f16e41928ff687:0x883dad4fd048e8f8,<NA>,32.388300,-83.357100,[Pharmacy],4.9,16,<NA>,"[[Friday, 8AM–6PM], [Saturday, 8AM–12PM], [Sun...",{'Accessibility': ['Wheelchair accessible entr...,Open ⋅ Closes 6PM,"[0x88f16e41929435cf:0x5b2532a2885e9ef6, 0x88f1...",https://www.google.com/maps/place//data=!4m2!3...
1,City Textile,"City Textile, 3001 E Pico Blvd, Los Angeles, C...",0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,<NA>,34.018891,-118.215290,[Textile exporter],4.5,6,<NA>,None,None,Open now,"[0x80c2c624136ea88b:0xb0315367ed448771, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...
2,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",0x80c2c778e3b73d33:0xbdc58662a4a97d49,<NA>,34.058092,-118.292130,[Korean restaurant],4.4,18,<NA>,"[[Thursday, 6:30AM–6PM], [Friday, 6:30AM–6PM],...",{'Accessibility': ['Wheelchair accessible entr...,Open ⋅ Closes 6PM,"[0x80c2c78249aba68f:0x35bf16ce61be751d, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...
3,Nova Fabrics,"Nova Fabrics, 2200 E 11th St, Los Angeles, CA ...",0x80c2c89923b27a41:0x32041559418d447,<NA>,34.023669,-118.232930,[Fabric store],3.3,6,<NA>,"[[Thursday, 9AM–5PM], [Friday, 9AM–5PM], [Satu...","{'Accessibility': None, 'Activities': None, 'A...",Open ⋅ Closes 5PM,"[0x80c2c8811477253f:0x23a8a492df1918f7, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...
4,Nobel Textile Co,"Nobel Textile Co, 719 E 9th St, Los Angeles, C...",0x80c2c632f933b073:0xc31785961fe826a6,<NA>,34.036694,-118.249421,[Fabric store],4.3,7,<NA>,"[[Thursday, 9AM–5PM], [Friday, 9AM–5PM], [Satu...","{'Accessibility': None, 'Activities': None, 'A...",Open ⋅ Closes 5PM,"[0x80c2c62c496083d1:0xdefa11317fe870a1, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...


In [5]:
# Tomamos las columnas que nos seran utiles
df_2 = df[['name','gmap_id','latitude','longitude','category','avg_rating','num_of_reviews','address']]

In [6]:
# Lo transformamos a un Data frame de Pandas
df_3 = pd.DataFrame(df_2)

In [7]:
# Renombramos las columnas
df_3.rename(columns={0:'name', 1:'gmap_id', 2:'latitude', 3:'longitude', 4:'category', 5:'avg_rating', 6:'num_of_reviews', 7:'address'}, inplace=True)

In [8]:
# Función para convertir la cadena en una lista
def str_to_list(s):
    if pd.isna(s):
        return []  # Manejar valores nulos devolviendo una lista vacía
    # Reemplaza los saltos de línea y espacios después de comillas simples con comas
    s = s.replace('\n', '').replace("' '", "','")

    # Convierte la cadena a una lista de Python
    return ast.literal_eval(s)

In [9]:
# Aplicamos la funcion a la columna 'category'
df_3['category'] = df_3['category'].apply(str_to_list)

In [10]:
# Extraemos todas las categorias por restaurante
df_4 = df_3.explode('category').reset_index(drop=True)

In [73]:
# Verificamos que este de forma correcta
df_4

,name,gmap_id,latitude,longitude,category,avg_rating,num_of_reviews,address
0,Porter Pharmacy,0x88f16e41928ff687:0x883dad4fd048e8f8,32.3883,-83.3571,Pharmacy,4.9,16,"Porter Pharmacy, 129 N Second St, Cochran, GA ..."
1,City Textile,0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,34.018891,-118.21529,Textile exporter,4.5,6,"City Textile, 3001 E Pico Blvd, Los Angeles, C..."
2,San Soo Dang,0x80c2c778e3b73d33:0xbdc58662a4a97d49,34.058092,-118.29213,Korean restaurant,4.4,18,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ..."
3,Nova Fabrics,0x80c2c89923b27a41:0x32041559418d447,34.023669,-118.23293,Fabric store,3.3,6,"Nova Fabrics, 2200 E 11th St, Los Angeles, CA ..."
4,Nobel Textile Co,0x80c2c632f933b073:0xc31785961fe826a6,34.036694,-118.249421,Fabric store,4.3,7,"Nobel Textile Co, 719 E 9th St, Los Angeles, C..."
...,...,...,...,...,...,...,...,...
6202452,Profiles Hair Design,0x8758dd1e2533c991:0x5f6d112918f07dba,43.029664,-108.380849,Beauty salon,4.7,76,"Profiles Hair Design, 522 N Federal Blvd, Rive..."
6202453,Profiles Hair Design,0x8758dd1e2533c991:0x5f6d112918f07dba,43.029664,-108.380849,Hair salon,4.7,76,"Profiles Hair Design, 522 N Federal Blvd, Rive..."
6202454,Arizona LINE-X,0x872b6f6f24aea445:0x22df9e5dd01e67ba,33.680066,-112.113367,Truck accessories store,4.6,24,"Arizona LINE-X, 21242 N Black Canyon Hwy #A, P..."
6202455,Arizona LINE-X,0x872b6f6f24aea445:0x22df9e5dd01e67ba,33.680066,-112.113367,Powder coating service,4.6,24,"Arizona LINE-X, 21242 N Black Canyon Hwy #A, P..."


In [11]:
restaurant_filter = df_4['category'].str.contains('restaurant', case=False, na=False) 
restaurantes = df_4[restaurant_filter]
restaurantes

,name,gmap_id,latitude,longitude,category,avg_rating,num_of_reviews,address
2,San Soo Dang,0x80c2c778e3b73d33:0xbdc58662a4a97d49,34.058092,-118.29213,Korean restaurant,4.4,18,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ..."
6,Vons Chicken,0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,33.916402,-118.010855,Restaurant,4.5,18,"Vons Chicken, 12740 La Mirada Blvd, La Mirada,..."
10,"Sweet Rewards Gluten Free Bakery, LLC",0x87ec235c54d25b31:0x3b75fb5facc602f,41.616079,-93.865487,Health food restaurant,4.7,21,"Sweet Rewards Gluten Free Bakery, LLC, 85 NE D..."
143,Vivi Bubble Tea,0x89c6c89efcaed69d:0xded973f6033e7dba,39.940293,-75.150923,Restaurant,4.0,8,"Vivi Bubble Tea, 701 S 5th St, Philadelphia, P..."
202,Hale Pops,0x7c00456eecad3111:0x8217f9600c51f33,21.637796,-157.920714,Restaurant,4.4,18,"Hale Pops, 55-370 Kamehameha Hwy, Laie, HI 96762"
...,...,...,...,...,...,...,...,...
6202443,Rosa’s Mexican Grill Chandler,0x872ba99db0ab78db:0x55c826ed8748149d,33.351057,-111.842995,Mexican restaurant,4.4,88,"Rosa’s Mexican Grill Chandler, 3002 N Arizona ..."
6202445,Domino's Pizza,0x8758dd6a49d17909:0xbe7ccb53ee53e5a7,43.031526,-108.381641,Delivery Restaurant,4.1,178,"Domino's Pizza, 804 N Federal Blvd, Riverton, ..."
6202446,Domino's Pizza,0x8758dd6a49d17909:0xbe7ccb53ee53e5a7,43.031526,-108.381641,Takeout Restaurant,4.1,178,"Domino's Pizza, 804 N Federal Blvd, Riverton, ..."
6202447,Domino's Pizza,0x8758dd6a49d17909:0xbe7ccb53ee53e5a7,43.031526,-108.381641,Pizza restaurant,4.1,178,"Domino's Pizza, 804 N Federal Blvd, Riverton, ..."


In [20]:
restaurantes['category'].value_counts()

category
Restaurant                                                                     97239
Fast food restaurant                                                           28798
Takeout Restaurant                                                             23912
Pizza restaurant                                                               23812
Mexican restaurant                                                             17203
                                                                               ...  
RestaurantWomen's clothing store                                                   1
Event venueChildren's party serviceRestaurant                                      1
Farmers' marketTaco restaurant                                                     1
Asturian restaurant                                                                1
FestivalMen's clothing storeMiddle Eastern restaurantWomen's clothing store        1
Name: count, Length: 354, dtype: int64

In [14]:
comida_rapida = restaurantes[restaurantes['category'] == 'Fast food restaurant']
comida_rapida.groupby('category')['avg_rating'].mean()

category
Fast food restaurant    3.821043
Name: avg_rating, dtype: object

In [15]:
comida_para_llevar = restaurantes[restaurantes['category'] == 'Takeout Restaurant']
comida_para_llevar.groupby('category')['avg_rating'].mean()

category
Takeout Restaurant    3.890775
Name: avg_rating, dtype: object

In [16]:
pizza = restaurantes[restaurantes['category'] == 'Pizza restaurant']
pizza.groupby('category')['n'].mean()

category
Pizza restaurant    4.030976
Name: avg_rating, dtype: object

In [17]:
mexican = restaurantes[restaurantes['category'] == 'Mexican restaurant']
mexican.groupby('category')['avg_rating'].mean()

category
Mexican restaurant    4.207475
Name: avg_rating, dtype: object

In [12]:
# Filtramos por el rubro que escogimos el cual fue restaurantes
df_filtrado = df_4[df_4["category"] == "Mexican restaurant"]

In [13]:
df_filtrado.reset_index(drop=True, inplace=True)

In [20]:
# Eliminamos los valores nulos
df_filtrado.dropna(inplace=True)

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_16684\1782011620.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado.dropna(inplace=True)


In [21]:
# Verificamos los tipos de datos por columna para ver si hay que cambiar algun tipo de dato
df_filtrado.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17143 entries, 0 to 17202
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   name            17143 non-null  object
 1   gmap_id         17143 non-null  object
 2   latitude        17143 non-null  object
 3   longitude       17143 non-null  object
 4   category        17143 non-null  object
 5   avg_rating      17143 non-null  object
 6   num_of_reviews  17143 non-null  object
 7   address         17143 non-null  object
dtypes: object(8)
memory usage: 1.2+ MB


In [22]:
# Creamos una funcion para extraer la ciudad desde la direccion
def extraer_ciudad(direccion):

    try:
        # Dividir la dirección en partes usando la coma como separador
        partes = direccion.split(", ")

        # Asegurarse de que haya al menos tres partes (ciudad, estado_código postal)
        if len(partes) >= 3:
            ciudad = partes[2]  # Extraer la 3ra parte como ciudad
            return ciudad
        else:
            return None # Devolver None para ciudad y estado si tiene la configuración planteada
    except IndexError:
        return None  # Devolver None para ciudad y estado si la extracción falla

In [23]:
# Agregamos una columna para colocar la ciudad
df_filtrado["city"] = df_filtrado['address'].apply(extraer_ciudad)

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_16684\2889900678.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado["city"] = df_filtrado['address'].apply(extraer_ciudad)


In [24]:
# Eliminamos la columna de direccion, ya que la ubicacion exacta la obtenemos con la latitud y longitud
df_filtrado.drop(columns='address', inplace=True)

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_16684\567693580.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado.drop(columns='address', inplace=True)


In [25]:
# Cambiamos el tipo de dato de las columnas que lo requieren para su correcto funcionamiento
df_filtrado['latitude'] = df_filtrado['latitude'].astype(float)
df_filtrado['longitude'] = df_filtrado['longitude'].astype(float)
df_filtrado['avg_rating'] = df_filtrado['avg_rating'].astype(float)
df_filtrado['num_of_reviews'] = df_filtrado['num_of_reviews'].astype(int)

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_16684\833960294.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado['latitude'] = df_filtrado['latitude'].astype(float)
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_16684\833960294.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado['longitude'] = df_filtrado['longitude'].astype(float)
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_16684\833960294.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

In [26]:
# Realizamos una ultima validacion para verificar que los tipos de datos sean correctos y que no hayan valores nulos
df_filtrado.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17143 entries, 0 to 17202
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   name            17143 non-null  object 
 1   gmap_id         17143 non-null  object 
 2   latitude        17143 non-null  float64
 3   longitude       17143 non-null  float64
 4   category        17143 non-null  object 
 5   avg_rating      17143 non-null  float64
 6   num_of_reviews  17143 non-null  int32  
 7   city            17141 non-null  object 
dtypes: float64(3), int32(1), object(4)
memory usage: 1.1+ MB


In [27]:
# Exportamos el dataset en formato parquet
df_filtrado.to_parquet('../Data/Parquet/Sitios.parquet', index=False)

## Reviews por estado
A continuacion se realizara la union de los distintos datasets que se nos brindaron para tener en un solo archivo los datos de los reviews y filtrado por la categoria de negocio elegida ('Restaurant')

In [2]:
# Guardamos como variables los directorios de los archivos que vamos a leer y a donde los queremos guardar
ruta_entrada = '../Data/Data_limpia/Google/Reviews'
ruta_salida = '../Data/Data_transformada/Estados'

# Iteramos el directorio para poder abrir cada carpeta de cada estado

for carpeta in os.listdir(ruta_entrada):
    # guardamos la ruta de la carpeta
    path_carpeta = os.path.join(ruta_entrada,carpeta)

    # Verificamos que sea una carpeta
    if os.path.isdir(path_carpeta):

        # Creamos una lista vacia que va a almacenar todos los data frames 
        dfs = []

        # Iteramos la carpeta para leer los archivos
        for item in os.listdir(path_carpeta):
            # Verificamos que sea un archivo JSON y guardamos su ruta
            if item.endswith('.json'):
                path_item = os.path.join(path_carpeta,item)

                # Leemos el archivo JSON
                df = pd.read_json(path_item, lines = True)

                # Añadimos el DF a la lista de DFS
                dfs.append(df)

        # Concatenamos los dataframes para tener un archivo unico      
        df_combinado = pd.concat(dfs) 

        # Definimos la ruta de salida
        salida = os.path.join(ruta_salida, f'{carpeta}.parquet')

        # Guardamos el data frame en formato parquet
        df_combinado.to_parquet(salida, index = False)

In [3]:
# Leemos uno de los datasets extraidos para verificar que este de la forma correcta
df = pd.read_parquet('../Data/Data_transformada/Estados/review-Alabama.parquet')
df.head(5)

,user_id,name,time,rating,text,pics,resp,gmap_id
0,1.140438e+20,Kanisha Mixon,1597168272670,5,Very Personable staff! Beautiful and clean env...,None,None,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf
1,1.160090e+20,Brandie Hodges,1609899039594,5,Best clothing intown,None,None,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf
2,1.062399e+20,Sharon King,1547235290843,4,None,None,None,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf
3,1.049701e+20,Veronica Pierce,1517709403534,5,None,None,None,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf
4,1.105875e+20,Whitney Waldon Collier,1535245718492,5,None,None,None,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf


In [28]:
ruta_entrada = '../Data/Data_transformada/Estados'
ruta_salida = '../Data/Data_transformada/Filtrados'

# Ruta que contiene los sitios
sitios_metadata = "../Data/Parquet/Sitios.parquet"

df_sitios = pd.read_parquet(sitios_metadata)

# Iteramos el directorio donde se encuentran los archivos antes extraidos
for item in os.listdir(ruta_entrada):

    # Cverificamos que sea un archivo parquet
    if item.endswith(".parquet"):
        # Leemos el archivo con Pandas
        df = pd.read_parquet(os.path.join(ruta_entrada, item))
        
        # Funcion para transformar en 'yes' si tiene datos la celda o en 'no' si esta vacia 
        def replace_none(value):
            if value is None:
                return "No"
            else:
                return "Yes"
            
        # Aplicamos la funcion en la columna 'resp' para saber si tiene respuesta el review o no
        df["resp"] = df["resp"].apply(replace_none)
        
        # Eliminamos la columna 'pics' ya que no nos proporciona informacion valiosa
        df.drop(["pics"], axis=1, inplace=True)
        
        # Eliminamos los duplicados
        df.drop_duplicates(inplace=True)

        # Renombramos la columna 'name' la cual contiene el nombre del usuario
        df.rename(columns={"name": "user_name"}, inplace = True)

        # Aplicamos un Merge con el dataset de los sitios para hacer el filtro de los restaurantes
        df = df.merge(
            df_sitios[["gmap_id", "name", "category", "city"]],
            on="gmap_id",
            how="left",
        )

        # Renombramos la columna 'name' la cual pertenece al nombre de la tienda
        df.rename(columns={"name": 'business_name'}, inplace = True)

        # Eliminamos valores nulos en las columnas importantes
        df = df.dropna(subset=["business_name", "city"])

        # Estraemos el estado del nombre del archivo parquet
        state_name = item.split("-")[-1].split(".")[0]

        # Agregamos una columna que especifica el nombre del estado en el que se realizo el review
        df["State_review"] = state_name

        # Configuramos el directorio de salida para cada archivo y se exporta en tipo parquet
        output_file_path = os.path.join(ruta_salida, item)
        df.to_parquet(output_file_path, index=False)


Despues de tener los archivos listos por estado, hacemos un pequeño analisis para ver que estado tendremos en cuenta segun la cantidad de reviews de restaurantes

In [29]:
# Ruta de los archivos ya filtrados
ruta_entrada = '../Data/Data_transformada/Filtrados'

# Creamos una lista vacia para almacenar los dataframes
dfs = []

# Iteramos la carpeta
for item in os.listdir(ruta_entrada):
    # Verificamos que sea un archivo parquet
    if item.endswith(".parquet"):
        # RLeemos el parquet por medio de Pandas
        df = pd.read_parquet(os.path.join(ruta_entrada, item))

        # Agregamos el Dataframe a la lista
        dfs.append(df)

# Concatenamos todos los datos en un solo dataframe para poder compararlos
df_Google_reviews_CS = pd.concat(dfs, ignore_index=True)

In [30]:
# Vemos la cantidad de registros
df_Google_reviews_CS.shape

(1100976, 11)

In [31]:
# Contamos la cantidad de registros por estado
df_Google_reviews_CS['State_review'].value_counts()

State_review
California              123218
Texas                    96517
Florida                  67325
Illinois                 61815
Arizona                  51188
Georgia                  46830
New_York                 41755
Colorado                 37502
Michigan                 35934
Ohio                     33053
North_Carolina           30444
Oregon                   29485
Indiana                  28245
Washington               28031
Virginia                 25057
Utah                     24518
Tennessee                23242
Nevada                   22045
Missouri                 21524
Wisconsin                20822
Oklahoma                 19363
New_Jersey               19348
New_Mexico               17214
South_Carolina           17141
Louisiana                16363
Pennsylvania             15934
Arkansas                 15106
Kansas                   14205
Kentucky                 13599
Iowa                     12918
Minnesota                12345
Maryland                 1

Despues de este pequeño analisis decidimos escoger el estado de Florida como enfoque, por lo tanto, realizaremos los ultimos detalles para por ultimo, exportar el archivo.

In [82]:
df_Google_reviews_CS['business_name'].value_counts()

business_name
Taco Bell                                  1
Crema Restaurante                          1
Sabor Latino                               1
Lucky Luna                                 1
Puebla Corner Deli                         1
                                          ..
El Toro Mexican Restaurant                 1
Picante American & Mexican Grill           1
Raymond's Taco's no.3                      1
Taco's El Norte                            1
Fiesta Guadalajara | Mexican Restaurant    1
Name: count, Length: 8150, dtype: int64

In [59]:
prueba = df_Google_reviews_CS['business_name'].unique()
prueba1 = df_Google_reviews_CS

In [64]:
prueba1.drop_duplicates(subset='business_name', inplace=True)

In [65]:
prueba1['State_review'].value_counts()

State_review
California              1118
Texas                    862
Illinois                 545
New_York                 517
Arizona                  274
Georgia                  262
Washington               249
New_Jersey               243
Colorado                 241
North_Carolina           218
Oregon                   215
Florida                  211
Indiana                  172
Tennessee                167
Oklahoma                 154
Ohio                     143
Virginia                 141
Arkansas                 139
Pennsylvania             136
Nevada                   131
New_Mexico               128
Missouri                 127
Michigan                 126
Kentucky                 121
Kansas                   115
Wisconsin                115
Alabama                  109
Utah                     106
Louisiana                106
Iowa                     105
Maryland                 104
South_Carolina           102
Massachusetts            102
Minnesota                 88
C

In [15]:
# Leemos el archivo correspondiente al estado de Florida
florida_df = pd.read_parquet('../Data/Data_transformada/Filtrados/review-Florida.parquet')
califronia_df = pd.read_parquet('../Data/Data_transformada/Filtrados/review-California.parquet')

In [24]:
florida_df.groupby('business_name')['rating'].mean().sort_values(ascending=True)

business_name
El Norteno                                3.071429
K' Rico's Restaurant                      3.200000
Down N' Dirty Tacos                       3.333333
Taco 71                                   3.363636
Mexico Bravo Now MEX Mexican Grill        3.384615
                                            ...   
Rincon Tolteca Mexican restaurant corp    4.954545
Taqueria El Molcajete                     5.000000
EL CAMION DEL TACO                        5.000000
Red Roc Cravings                          5.000000
TacoSchool El Compadre                    5.000000
Name: rating, Length: 229, dtype: float64

In [68]:
# Revisamos los tipos de datos y los valores nulos
florida_df.info()
califronia_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67325 entries, 0 to 67324
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   user_id        67325 non-null  float64
 1   user_name      67325 non-null  object 
 2   time           67325 non-null  int64  
 3   rating         67325 non-null  int64  
 4   text           39360 non-null  object 
 5   resp           67325 non-null  object 
 6   gmap_id        67325 non-null  object 
 7   business_name  67325 non-null  object 
 8   category       67325 non-null  object 
 9   city           67325 non-null  object 
 10  State_review   67325 non-null  object 
dtypes: float64(1), int64(2), object(8)
memory usage: 5.7+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123218 entries, 0 to 123217
Data columns (total 11 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   user_id        123218 non-null  float64
 1   user_n

In [69]:
# Creamos una funcion que permite extraer la fecha y la hora desde la columna de time

def convert_timestamp_ms_to_datetime(timestamp_ms):
    """
    Convierte un timestamp en milisegundos a un objeto datetime.
    """
    if pd.isna(timestamp_ms):
        return None
    # Convertir a segundos
    timestamp_s = timestamp_ms / 1000.0
    # Convertir a datetime
    return datetime.fromtimestamp(timestamp_s)

In [70]:
# Aplicamos la funcion a la columna de time
florida_df['time'] = florida_df['time'].apply(convert_timestamp_ms_to_datetime)

In [71]:
# Creamos las columnas de fecha, hora y dia de la semana a partir de la fecha extraida anteriormente
florida_df['date'] = florida_df['time'].dt.normalize()
florida_df['hour'] = florida_df['time'].dt.hour
florida_df['day'] = florida_df['time'].dt.day_name()

In [72]:
# Verificamos que este correcto
florida_df.head(5)

,user_id,user_name,time,rating,text,resp,gmap_id,business_name,category,city,State_review,date,hour,day
0,1.176490e+20,Jane Cook,2015-05-25 17:56:04.403,5,any time we are in Pompano Beach we go for lun...,No,0x88d90319155c44a9:0x9abfade11830d0ca,Starlight Restaurant & Lounge,Mexican restaurant,Pompano Beach,Florida,2015-05-25,17,Monday
1,1.041909e+20,Mark Felix,2017-11-14 09:40:32.156,4,Good service. Food was so so.,No,0x88d90319155c44a9:0x9abfade11830d0ca,Starlight Restaurant & Lounge,Mexican restaurant,Pompano Beach,Florida,2017-11-14,9,Tuesday
2,1.121549e+20,Yanio w Alfonso,2018-08-19 06:48:00.000,1,Nasty place and very expensive.,No,0x88d90319155c44a9:0x9abfade11830d0ca,Starlight Restaurant & Lounge,Mexican restaurant,Pompano Beach,Florida,2018-08-19,6,Sunday
3,1.126933e+20,Carlos San Luis,2018-01-01 15:27:14.949,5,Good mexican food,No,0x88d90319155c44a9:0x9abfade11830d0ca,Starlight Restaurant & Lounge,Mexican restaurant,Pompano Beach,Florida,2018-01-01,15,Monday
4,1.131080e+20,Maigra Hernandez,2018-03-04 15:17:22.630,5,Tacos were great,No,0x88d90319155c44a9:0x9abfade11830d0ca,Starlight Restaurant & Lounge,Mexican restaurant,Pompano Beach,Florida,2018-03-04,15,Sunday


In [73]:
# Eliminamos la columna time, ya que los datos los extraimos en otras columnas
florida_df.drop(columns={'time'}, inplace = True)

In [45]:
# Eliminamos los valores nulos ya que el dataset aun es muy grande y necesitamos reducir el volumen de datos
florida_df.dropna(inplace=True)
florida_df.reset_index(inplace=True)

In [74]:
# Validar que los tipos de datos esten correctos
florida_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67325 entries, 0 to 67324
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   user_id        67325 non-null  float64       
 1   user_name      67325 non-null  object        
 2   rating         67325 non-null  int64         
 3   text           39360 non-null  object        
 4   resp           67325 non-null  object        
 5   gmap_id        67325 non-null  object        
 6   business_name  67325 non-null  object        
 7   category       67325 non-null  object        
 8   city           67325 non-null  object        
 9   State_review   67325 non-null  object        
 10  date           67325 non-null  datetime64[ns]
 11  hour           67325 non-null  int32         
 12  day            67325 non-null  object        
dtypes: datetime64[ns](1), float64(1), int32(1), int64(1), object(9)
memory usage: 6.4+ MB


In [75]:
# Vemos el rango de fechas que tenemos para el analisis

fecha_minima = florida_df['date'].min()
fecha_maxima = florida_df['date'].max()

print(f'EL rango de fechas para el analisis va desde el {fecha_minima} hasta el {fecha_maxima}')

EL rango de fechas para el analisis va desde el 2009-05-11 00:00:00 hasta el 2021-08-30 00:00:00


In [76]:
# Exportamos el dataset en un archivo tipo parquet
florida_df.to_parquet('../Data/Parquet/Reviews_florida.parquet', index=False)

In [77]:
califronia_df['time'] = califronia_df['time'].apply(convert_timestamp_ms_to_datetime)

In [78]:
# Creamos las columnas de fecha, hora y dia de la semana a partir de la fecha extraida anteriormente
califronia_df['date'] = califronia_df['time'].dt.normalize()
califronia_df['hour'] = califronia_df['time'].dt.hour
califronia_df['day'] = califronia_df['time'].dt.day_name()

In [79]:
califronia_df.head(5)

,user_id,user_name,time,rating,text,resp,gmap_id,business_name,category,city,State_review,date,hour,day
0,1.089843e+20,Edgar Alexander Lara,2021-04-13 00:03:26.946,4,A delicious variety of food. Good place to go ...,No,0x80c2d765f8c90a3d:0x16afb75943e7ad50,Cowboy Burgers & BBQ,Mexican restaurant,Baldwin Park,California,2021-04-13,0,Tuesday
1,1.089843e+20,Edgar Alexander Lara,2021-04-13 00:03:26.946,4,A delicious variety of food. Good place to go ...,No,0x80c2d765f8c90a3d:0x16afb75943e7ad50,Cowboy Burgers & BBQ,Mexican restaurant,Baldwin Park,California,2021-04-13,0,Tuesday
2,1.022832e+20,Selena Mejia,2014-06-15 21:13:10.840,1,I use to love cowboys ever since I was a teen....,No,0x80c2d765f8c90a3d:0x16afb75943e7ad50,Cowboy Burgers & BBQ,Mexican restaurant,Baldwin Park,California,2014-06-15,21,Sunday
3,1.022832e+20,Selena Mejia,2014-06-15 21:13:10.840,1,I use to love cowboys ever since I was a teen....,No,0x80c2d765f8c90a3d:0x16afb75943e7ad50,Cowboy Burgers & BBQ,Mexican restaurant,Baldwin Park,California,2014-06-15,21,Sunday
4,1.113197e+20,Roderick Conwi,2016-02-21 16:32:39.368,4,If you like going to mom and pop burger joints...,Yes,0x80c2d765f8c90a3d:0x16afb75943e7ad50,Cowboy Burgers & BBQ,Mexican restaurant,Baldwin Park,California,2016-02-21,16,Sunday


In [80]:
califronia_df.drop(columns={'time'}, inplace = True)

In [81]:
# Vemos el rango de fechas que tenemos para el analisis

fecha_minima = califronia_df['date'].min()
fecha_maxima = califronia_df['date'].max()

print(f'EL rango de fechas para el analisis va desde el {fecha_minima} hasta el {fecha_maxima}')

EL rango de fechas para el analisis va desde el 2003-10-30 00:00:00 hasta el 2021-09-03 00:00:00


In [83]:
# Exportamos el dataset en un archivo tipo parquet
califronia_df.to_parquet('../Data/Parquet/Reviews_california.parquet', index=False)